## Load Document $\rightarrow$ Split Document $\rightarrow$ Storage $\rightarrow$ retrieval $\rightarrow$ **Output**

![Steps](image.jpg)

In [2]:
from langchain.document_loaders import PyPDFLoader
from pprint import pprint
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
llm_name = "gpt-3.5-turbo"

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [6]:
print(vectordb._collection.count())

209


In [7]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [8]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [9]:
pretty_print_docs(docs)

Document 1:

statistics for a while or maybe algebra, we'll go over those in the discussion sections as a 
refresher for those of you that want one.  
Later in this quarter, we'll also use the disc ussion sections to go over extensions for the 
material that I'm teaching in the main lectur es. So machine learning is a huge field, and 
there are a few extensions that we really want  to teach but didn't have time in the main 
lectures for.
----------------------------------------------------------------------------------------------------
Document 2:

statistics for a while or maybe algebra, we'll go over those in the discussion sections as a 
refresher for those of you that want one.  
Later in this quarter, we'll also use the disc ussion sections to go over extensions for the 
material that I'm teaching in the main lectur es. So machine learning is a huge field, and 
there are a few extensions that we really want  to teach but didn't have time in the main 
lectures for.
---------------

In [10]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [10]:
# from langchain import HuggingFaceHub

# llm=HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"temperature":0.1 ,"max_length":512})

### retrievalqachain

In [6]:
from langchain.chains import RetrievalQA

In [12]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [13]:
result = qa_chain({"query": question})

In [14]:
result

{'query': 'What are major topics for this class?',
 'result': 'The major topics for this class are machine learning and its various extensions.'}

In [15]:
result["result"]

'The major topics for this class are machine learning and its various extensions.'

### prompt

In [2]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [10]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

NameError: name 'vectordb' is not defined

In [4]:
question = "Is probability a class topic?"

In [5]:
result = qa_chain({"query": question})

NameError: name 'qa_chain' is not defined

In [26]:
result["result"]

'The major topics for this class are machine learning and its various extensions.'

In [25]:
result["source_documents"][0]

Document(page_content="of this class will not be very program ming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octa ve. I'll say a bit more about that later.  \nI also assume familiarity with basic proba bility and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what ra ndom variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites, sort of as  a refresher course under prerequisite class. \nI'll say a bit more about that later as well.  \nLastly, I also assume familiarity with basi c linear algebra. And again, most undergraduate \nlinear algebra courses are more than enough. So if you've taken courses like Math 51, \n103, Math 113 or 

In [29]:
question_2= "Is monday a day of the week?"

In [30]:
result = qa_chain({"query": question_2})


In [31]:
result["result"]

"I don't know. Thanks for asking!"

### RetrievalQA chain types

#### Map reduce

In [26]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type='mmr'),
    return_source_documents=True,
    chain_type="map_reduce"
)

In [32]:
# question = "What are major topics for this class?"


In [35]:
question = "Is probability a class topic?"

In [36]:
result = qa_chain_mr({"query": question})

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-OKEaxoW73Lo9pocIXi1wDApB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-OKEaxoW73Lo9pocIXi1wDApB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

In [37]:
result["result"]

'Yes, probability is mentioned as a prerequisite for the class. The instructor assumes familiarity with basic probability and statistics.'

#### Refine

In [38]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

"Yes, probability is commonly taught as a class topic in various educational settings. It is typically included as a standalone course or as part of a broader mathematics or statistics curriculum. Probability classes cover fundamental principles and techniques used to quantify uncertainty and make predictions in different scenarios.\n\nIn a probability class, students learn about basic probability rules, conditional probability, independence, random variables, probability distributions (including discrete and continuous distributions), expected value, variance, and the central limit theorem. They also explore various probability models and their applications in real-world situations.\n\nThe class often involves practical exercises and problem-solving to enhance students' analytical and critical thinking skills. Students may also gain experience using software or programming languages to simulate and analyze probabilistic scenarios.\n\nOverall, probability is a crucial topic in mathemat

### RetrievalQA limitations

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

In [ ]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]